* explicit getter and setter methods in their classes
 - Using these setters and getters is simple, but it’s not Pythonic

In [1]:
import logging
from pprint import pprint
from sys import stdout as STDOUT


# Example 1
class OldResistor(object):
    def __init__(self, ohms):
        self._ohms = ohms

    def get_ohms(self):
        return self._ohms

    def set_ohms(self, ohms):
        self._ohms = ohms


# Example 2
r0 = OldResistor(50e3)
print('Before: %5r' % r0.get_ohms())
r0.set_ohms(10e3)
print('After:  %5r' % r0.get_ohms())


# Example 3
r0.set_ohms(r0.get_ohms() + 5e3)


Before: 50000.0
After:  10000.0


In Python, however, you almost never need to implement explicit setter or getter methods.
Instead, you should always start your implementations with simple public attributes.


In [2]:
# Example 4
class Resistor(object):
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0

r1 = Resistor(50e3)
r1.ohms = 10e3
print('%r ohms, %r volts, %r amps' %
      (r1.ohms, r1.voltage, r1.current))


# Example 5
r1.ohms += 5e3



10000.0 ohms, 0 volts, 0 amps


Later, if you decide you need special behavior when an attribute is set, you can migrate to
the @property decorator and its corresponding setter attribute.
Note that in order to work properly the name of both the setter and getter
methods must match the intended property name.


In [5]:
# Example 6
class VoltageResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0

    @property
    def voltage(self):
        return self._voltage

    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms


# Example 7
r2 = VoltageResistance(1e3)
print('Before: %5r amps' % r2.current)
r2.voltage = 10
print('After:  %5r amps' % r2.current)



Before:     0 amps
After:   0.01 amps


In [16]:
# Example 8
class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)

    @property
    def ohms(self):
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError('%f ohms must be > 0' % ohms)
        self._ohms = ohms


# Example 9
try:
    r3 = BoundedResistance(1e3)
    r3.ohms = 0
except:
    logging.exception('Expected')
else:
    assert False



# Example 10
try:
    BoundedResistance(-5)
except:
    logging.exception('Expected')
else:
    assert False



ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-16-71dca559e2b4>", line 20, in <module>
    r3.ohms = 0
  File "<ipython-input-16-71dca559e2b4>", line 13, in ohms
    raise ValueError('%f ohms must be > 0' % ohms)
ValueError: 0.000000 ohms must be > 0
ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-16-71dca559e2b4>", line 30, in <module>
    BoundedResistance(-5)
  File "<ipython-input-16-71dca559e2b4>", line 4, in __init__
    super().__init__(ohms)
  File "<ipython-input-2-1a735d1cdc26>", line 4, in __init__
    self.ohms = ohms
  File "<ipython-input-16-71dca559e2b4>", line 13, in ohms
    raise ValueError('%f ohms must be > 0' % ohms)
ValueError: -5.000000 ohms must be > 0


* You can even use @property to make attributes from parent classes immutable

In [17]:
# Example 11
class FixedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)

    @property
    def ohms(self):
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        if hasattr(self, '_ohms'):
            raise AttributeError("Can't set attribute")
        self._ohms = ohms


# Example 12
try:
    r4 = FixedResistance(1e3)
    r4.ohms = 2e3
except:
    logging.exception('Expected')
else:
    assert False



ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-17-7fa779bf0061>", line 20, in <module>
    r4.ohms = 2e3
  File "<ipython-input-17-7fa779bf0061>", line 13, in ohms
    raise AttributeError("Can't set attribute")
AttributeError: Can't set attribute


* The biggest shortcoming of @property is that the methods for an attribute can only be shared by subclasses. Unrelated classes can’t share the same implementation. However, Python also supports descriptors (see Item 31: “Use Descriptors for Reusable @property Methods”) that enable reusable property logic and many other use cases. Finally, when you use @property methods to implement setters and getters, be sure that the behavior you implement is not surprising. For example, don’t set other attributes in getter property methods.

In [18]:
class MysteriousResistor(Resistor):
    @property
    def ohms(self):
        self.voltage = self._ohms * self.current
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        self._ohms = ohms


# Example 14
r7 = MysteriousResistor(10)
r7.current = 0.01
print('Before: %5r' % r7.voltage)
r7.ohms
print('After:  %5r' % r7.voltage)



Before:     0
After:    0.1


* Define new class interfaces using simple public attributes, and avoid set and get methods.
* Use @property to define special behavior when attributes are accessed on your objects, if necessary.
* Follow the rule of least surprise and avoid weird side effects in your @property methods.
* Ensure that @property methods are fast; do slow or complex work using normal methods.